In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt

from matplotlib.colors import BoundaryNorm
import matplotlib.colors as colors
from scipy import signal
import pickle
from multiprocessing.pool import Pool

import demo_common
%matplotlib notebook

In [ ]:
#### create ram DISK
# sudo mkdir /mnt/tmpfs/
# sudo chmod 777 /mnt/tmpfs/
# sudo mount -t tmpfs -o size=300M tmpfs /mnt/tmpfs

## in /etc/fstab
#tmpfs /mnt/tmpfs tmpfs defaults,size=500M 0 0

### Get all MP3 files in path

In [47]:
mp3_files = []

def get_mp3_files(path):
    for path, dirs, files in os.walk(path):
        for f in files:
            if not f.endswith('.mp3'):
                continue
            mp3_file = os.path.join(path, f)
            mp3_files.append(mp3_file)
            
get_mp3_files('/home/oscil/Downloads/')

In [48]:
len(mp3_files)

611

In [49]:
def make_sum_pkl_for_file(file):
    out_file_name = demo_common.convert2wav(file, freq = 44100, mono=True)
    params, samples = demo_common.read_wav(out_file_name)
    os.remove(out_file_name)
    samples = samples[0]
    
    fft_samples = 2048
    fft_overlap = 1024
    
    f, t, Sxx = signal.spectrogram(samples, 
                                   params['framerate'], 
                                   nperseg = fft_samples, 
                                   noverlap = fft_overlap)
    
    chunk_size_sec = 30
    freq_chunks_in_second = 44100/(fft_samples - fft_overlap)
    
    spec_sums = []
    cur_sum = None
    last_chunk_t = 0
    for idx in range(len(t)):
        if (t[idx] - last_chunk_t)//chunk_size_sec:
            spec_sums.append( {'t':last_chunk_t, 's':cur_sum} )
            last_chunk_t = t[idx]
            cur_sum = None
        else:
            if cur_sum is None:
                cur_sum = Sxx.T[idx]
            else:
                cur_sum += Sxx.T[idx]
    out_dir = '/home/oscil/prepared_music_files/'
    out_dict = {'f':file, 'sums':spec_sums}
    
    out_file_name = out_file_name.split('/')[-1]
    out_file_name = out_file_name.split('.')[0]
    out_file_name = out_file_name[-min(len(out_file_name),80):]
    try:
        with open( out_dir + out_file_name+'_s.pkl', 'wb') as f:       
            pickle.dump(out_dict, f)
    except OSError as exc:
        if exc.errno == 36:
            print('Handled error FILE NAME TOO LONG for {}'.format(out_file_name) )

In [51]:
len(mp3_files)

611

In [50]:
pool = Pool(7)
zip(*pool.map(make_sum_pkl_for_file, mp3_files))

Handled error FILE NAME TOO LONG for _Оркестр Поля Мориа_1981 Оркестр Поля Мориа - Под музыку Вивальди_04 - Жаворонок
Handled error FILE NAME TOO LONG for оля Мориа_1981 Оркестр Поля Мориа - Под музыку Вивальди_01 - Под Музыку Вивальди
Handled error FILE NAME TOO LONG for Оркестр Поля Мориа_1981 Оркестр Поля Мориа - Под музыку Вивальди_12 - Бабье Лето
Handled error FILE NAME TOO LONG for Оркестр Поля Мориа_1981 Оркестр Поля Мориа - Под музыку Вивальди_14 - За Любовью
Handled error FILE NAME TOO LONG for Мориа_1981 Оркестр Поля Мориа - Под музыку Вивальди_05 - Пусть Нам Светит Солнце
Handled error FILE NAME TOO LONG for ркестр Поля Мориа_1981 Оркестр Поля Мориа - Под музыку Вивальди_07 - Приключения
Handled error FILE NAME TOO LONG for стр Поля Мориа_1981 Оркестр Поля Мориа - Под музыку Вивальди_03 - Любовь Грустна
Handled error FILE NAME TOO LONG for _Оркестр Поля Мориа_1974 Оркестр Поля Мориа - Музыка из кинофильмов_06 - Встреча
Handled error FILE NAME TOO LONG for тр Поля Мориа_1974 

TypeError: zip argument #1 must support iteration

In [ ]:
with open( out_file_name+'_sum.pkl', 'wb') as f:
    pickle.dump(spec_sums, f)

In [ ]:
mp3_files[0]

In [ ]:
# fig, ax = plt.subplots()
# pcm = ax.pcolormesh(t, f, Sxx, norm=colors.PowerNorm(gamma=1./5.) )
# fig.colorbar(pcm, ax=ax, extend='max')

In [ ]:
fig, ax = plt.subplots()
for idx in range(len(spec_sums)):
    ax.plot(np.linspace(0,44100/2,len(spec_sums[idx]['s'])),  spec_sums[idx]['s'], alpha=0.5)
